In [3]:
import pandas as pd

# Đường dẫn đến tệp dữ liệu CSV
file_path = 'NDVI_Pixels_Time_Series_All_Years_quehuong.csv'

# Đọc tệp CSV vào DataFrame
data = pd.read_csv(file_path)
data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  1200 non-null   object 
 1   NDVI          1200 non-null   float64
 2   year          1200 non-null   float64
 3   .geo          1200 non-null   object 
dtypes: float64(2), object(2)
memory usage: 37.6+ KB


In [4]:
data[['year_number', 'pixel_id']] = data['system:index'].str.split('_', expand=True)
data['year_number'] = data['year'].astype(int)
data['pixel_id'] = data['pixel_id'].astype(int)

# Chuyển đổi DataFrame để mỗi dòng là một pixel và mỗi cột là NBR của từng năm
pivot_df = data.pivot(index='pixel_id', columns='year_number', values='NDVI')

# Đổi tên các cột để phản ánh năm theo thứ tự (Year_1, Year_2, ..., Year_38)
pivot_df.columns = [f'Year_{col}' for col in pivot_df.columns]

# Hiển thị kết quả
pivot_df.reset_index(inplace=True)
pivot_df.head()
pivot_df.to_csv('pivot_data.csv', index=False)

In [5]:
import xarray as xr
data = pivot_df.dropna()
years = [int(col.split('_')[1]) for col in data.columns if col.startswith("Year_")]
pixel_ids = data['pixel_id']

# Chuyển đổi DataFrame sang xarray.Dataset
ds = xr.Dataset(
    {
        "NDVI": (["pixel_id", "year"], data.drop(columns="pixel_id").values)
    },
    coords={
        "pixel_id": pixel_ids,
        "time": years
    }
)

# Hiển thị Dataset
print(ds)

<xarray.Dataset>
Dimensions:   (pixel_id: 48, year: 25, time: 25)
Coordinates:
  * pixel_id  (pixel_id) int32 0 1 2 3 4 5 6 7 8 ... 39 40 41 42 43 44 45 46 47
  * time      (time) int32 1986 1987 1988 1989 1990 ... 2006 2007 2008 2009 2010
Dimensions without coordinates: year
Data variables:
    NDVI      (pixel_id, year) float64 0.2686 0.1902 0.123 ... 0.1047 0.2587


In [6]:
# Đọc tệp CSV vào DataFrame
data = pd.read_csv("pivot_data.csv")  
years = [int(col.split('_')[1]) for col in data.columns if col.startswith("Year_")]
pixel_ids = data['pixel_id']

ds = xr.Dataset(
    {
        "NDVI": (["pixel_id", "year"], data.drop(columns="pixel_id").values)
    },
    coords={
        "pixel_id": pixel_ids,
        "time": years
    }
)

In [1]:
from dask.distributed import Client

In [2]:
client = Client("tcp://127.0.0.1:5763")
print(client)  # Hiển thị thông tin cluster

<Client: 'tcp://10.244.0.14:8786' processes=3 threads=24, memory=22.97 GiB>


In [12]:
import tarfile
import rasterio

# Mở tệp tar
tar_file = 'data/LT05_L2SP_127045_19860701_20200917_02_T1.tar'
with tarfile.open(tar_file, 'r') as tar:
    tar.list()  # Liệt kê nội dung trong tar file
    tif_files = [file for file in tar.getnames() if file.endswith('.TIF')]
    for tif_file in tif_files:
        tar.extract(tif_file, path=f'data/LT05_L2SP_127045_19860701_20200917_02_T1')  # Giải nén từng tệp .tif


?rw-r--r-- root/root      34727 2024-11-27 14:20:05 LT05_L2SP_127045_19860701_20200917_02_T1_ANG.txt 
?rw-r--r-- root/root      13057 2024-11-27 14:20:06 LT05_L2SP_127045_19860701_20200917_02_T1_MTL.json 
?rw-r--r-- root/root      13988 2024-11-27 14:20:06 LT05_L2SP_127045_19860701_20200917_02_T1_MTL.txt 
?rw-r--r-- root/root      20476 2024-11-27 14:20:06 LT05_L2SP_127045_19860701_20200917_02_T1_MTL.xml 
?rw-r--r-- root/root    1960377 2024-11-27 14:20:06 LT05_L2SP_127045_19860701_20200917_02_T1_QA_PIXEL.TIF 
?rw-r--r-- root/root     454131 2024-11-27 14:20:06 LT05_L2SP_127045_19860701_20200917_02_T1_QA_RADSAT.TIF 
?rw-r--r-- root/root    7722913 2024-11-27 14:20:06 LT05_L2SP_127045_19860701_20200917_02_T1_SR_ATMOS_OPACITY.TIF 
?rw-r--r-- root/root   66736879 2024-11-27 14:20:08 LT05_L2SP_127045_19860701_20200917_02_T1_SR_B1.TIF 
?rw-r--r-- root/root   62353134 2024-11-27 14:20:10 LT05_L2SP_127045_19860701_20200917_02_T1_SR_B2.TIF 
?rw-r--r-- root/root   64664252 2024-11-27 14:20:11 L

In [14]:
import os
import tarfile
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
from rasterio.mask import mask

# Mở tệp tar và giải nén các tệp .TIF
tar_file = 'data/LT05_L2SP_127045_19860701_20200917_02_T1.tar'
with tarfile.open(tar_file, 'r') as tar:
    tif_files = [file for file in tar.getnames() if file.endswith('.TIF')]
    for tif_file in tif_files:
        tar.extract(tif_file, path=f'data/LT05_L2SP_127045_19860701_20200917_02_T1')  # Giải nén từng tệp .tif

# Tạo thư mục đầu ra cho COG nếu chưa tồn tại
output_dir = "output_cog"
os.makedirs(output_dir, exist_ok=True)

# Tạo Polygon từ tọa độ của bạn (tọa độ là danh sách các điểm [longitude, latitude])
coords = [
    (105.312544, 21.519839),
    (105.548750, 21.597749),
    (105.807615, 21.342149),
    (105.732084, 21.189216),
    (105.474040, 21.129934)
]
polygon = Polygon(coords)

# Chuyển Polygon thành GeoDataFrame (EPSG:4326 cho các tọa độ là lat, lon)
gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs="EPSG:4326")

# Tạo danh sách để lưu trữ các band
all_bands = []

# Duyệt qua các tệp .tif để đọc tất cả các band
for tif_file in tif_files:
    tif_path = f'data/LT05_L2SP_127045_19860701_20200917_02_T1/{tif_file}'
    
    with rasterio.open(tif_path) as src:
        # Kiểm tra CRS của ảnh và chuyển đổi CRS của polygon nếu cần
        if src.crs != "EPSG:4326":
            gdf = gdf.to_crs(src.crs)

        # Cắt ảnh theo polygon đã chuyển đổi CRS
        out_image, out_transform = mask(src, gdf.geometry, crop=True)
        
        # Lưu trữ band vào danh sách (mỗi band là một mảng NumPy)
        all_bands.append(out_image)

# Kiểm tra nếu tất cả các band đã được đọc
# Nếu có ít nhất 1 band, tiếp tục gộp và lưu
if all_bands:
    # Lấy metadata từ band đầu tiên để cập nhật
    with rasterio.open(f'data/LT05_L2SP_127045_19860701_20200917_02_T1/{tif_files[0]}') as src:
        out_meta = src.meta
    # Cập nhật metadata để phù hợp với tất cả các band
    out_meta.update({
        "driver": "GTiff", 
        "dtype": "uint16",  # Kiểu dữ liệu của ảnh
        "nodata": None,     # Nếu không có NoData, bạn có thể để None
        "count": len(all_bands),  # Số lượng band (nếu bạn có nhiều band)
        "crs": src.crs,     # CRS của ảnh (có thể là EPSG:32648 hoặc một CRS khác)
        "transform": out_transform,  # Biến đổi affine cho ảnh cắt
        "tiled": True,      # Tạo ảnh theo kiểu tiling
        "blockxsize": 512,  # Kích thước block theo chiều ngang
        "blockysize": 512,  # Kích thước block theo chiều dọc
        "compress": "lzw",  # Nén ảnh với LZW
        "predictor": 2,     # Dùng predictor cho ảnh có giá trị giống nhau
        "overviews": "YES", # Tạo overviews (ảnh thu nhỏ) cho ảnh
    })

    # Tạo đường dẫn mới để lưu ảnh COG
    output_file = os.path.join(output_dir, f'cog_multiband.tif')

    # Lưu ảnh với tất cả các band vào GeoTIFF (COG)
    with rasterio.open(output_file, 'w', **out_meta) as dest:
        for i in range(len(all_bands)):
            print(all_bands[i].shape)
            dest.write(all_bands[i], i + 1)  # Ghi tất cả các band vào tệp

    print(f"Đã tạo COG với nhiều band: {output_file}")
else:
    print("Không có band nào được đọc từ các tệp .TIF.")


(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)
(1, 1727, 1713)


ValueError: Source shape (1, 1, 1727, 1713) is inconsistent with given indexes 1

In [13]:
len(all_bands)

19